<a href="https://colab.research.google.com/github/Felgaba/HSE_DS_MasterTrack/blob/main/MLLib_and_structured_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q pyspark

     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 50.4 MB/s 


In [ ]:
! pip install -q findspark

In [ ]:
import findspark
findspark.init()
import pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip

--2022-11-24 06:50:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198.65K  --.-KB/s    in 0.1s    

2022-11-24 06:50:18 (1.77 MB/s) - ‘smsspamcollection.zip’ saved [203415/203415]



In [ ]:
!unzip smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!head SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [ ]:
src = (
    spark
    .read
    .option("sep", "\t")
    .csv("SMSSpamCollection")
    .withColumnRenamed("_c0", "label")
    .withColumnRenamed("_c1", "message")
)

In [ ]:
src.show()

+-----+--------------------+
|label|             message|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml import feature

In [ ]:
tokenizer = feature.Tokenizer(inputCol="message", outputCol="tokens") #sokit sentences by words

In [ ]:
tokenizer.transform(src).show()

+-----+--------------------+--------------------+
|label|             message|              tokens|
+-----+--------------------+--------------------+
|  ham|Go until jurong p...|[go, until, juron...|
|  ham|Ok lar... Joking ...|[ok, lar..., joki...|
| spam|Free entry in 2 a...|[free, entry, in,...|
|  ham|U dun say so earl...|[u, dun, say, so,...|
|  ham|Nah I don't think...|[nah, i, don't, t...|
| spam|FreeMsg Hey there...|[freemsg, hey, th...|
|  ham|Even my brother i...|[even, my, brothe...|
|  ham|As per your reque...|[as, per, your, r...|
| spam|WINNER!! As a val...|[winner!!, as, a,...|
| spam|Had your mobile 1...|[had, your, mobil...|
|  ham|I'm gonna be home...|[i'm, gonna, be, ...|
| spam|SIX chances to wi...|[six, chances, to...|
| spam|URGENT! You have ...|[urgent!, you, ha...|
|  ham|I've been searchi...|[i've, been, sear...|
|  ham|I HAVE A DATE ON ...|[i, have, a, date...|
| spam|XXXMobileMovieClu...|[xxxmobilemoviecl...|
|  ham|Oh k...i'm watchi...|[oh, k...i'm, wat...|


In [ ]:
from pyspark.ml import pipeline

In [ ]:
src.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  ham| 4827|
| spam|  747|
+-----+-----+



In [ ]:
from pyspark.ml import classification

In [ ]:
train, test = src.randomSplit(seed=1234, weights=(70., 30.))

In [ ]:
main_pipe = pipeline.Pipeline(
    stages=(
        feature.Tokenizer(inputCol="message", outputCol="tokens"),
        feature.CountVectorizer(
            maxDF=4000,
            minDF=5,
            inputCol="tokens", 
            outputCol="vector",
        ),
        feature.StringIndexer(inputCol="label", outputCol="y"),
        classification.NaiveBayes(featuresCol="vector", labelCol="y"),
    )
)

In [ ]:
pipe_model = main_pipe.fit(train)

In [ ]:
pipe_model.transform(test).show()

+-----+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+
|label|             message|              tokens|              vector|  y|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+
|  ham| &lt;#&gt;  in mc...|[, &lt;#&gt;, , i...|(1502,[7,20,24,26...|0.0|[-29.189065862648...|[0.99999999839701...|       0.0|
|  ham| &lt;#&gt;  mins ...|[, &lt;#&gt;, , m...|(1502,[0,1,20,24,...|0.0|[-57.645920593005...|[0.99999995129602...|       0.0|
|  ham| came to look at ...|[, came, to, look...|(1502,[0,4,7,8,20...|0.0|[-106.80303662568...|[0.99999999987274...|       0.0|
|  ham|&lt;#&gt;  w jett...|[&lt;#&gt;, , w, ...|(1502,[2,20,29,40...|0.0|[-34.942124728177...|[0.99999960047460...|       0.0|
|  ham|&lt;#&gt; ISH MIN...|[&lt;#&gt;, ish, ...|(1502,[40,58,408,...|0.0|[-34.609496567593...|[0.998687

In [ ]:
spam_stream_dir = "/stream/spam/"

!mkdir -p $spam_stream_dir

In [ ]:
# !ls /stream/spam/

In [ ]:
!shuf SMSSpamCollection | head -1000 > `tempfile -d $spam_stream_dir`

In [ ]:
src.schema

StructType([StructField('label', StringType(), True), StructField('message', StringType(), True)])

In [ ]:
spam_stream = (
    spark
    .readStream
    .schema(src.schema)
    .option("sep", "\t")
    .csv(spam_stream_dir)
)

In [ ]:
spam_stream.writeStream.format("memory").queryName("spam_stream").start() #write spark table in the temporal SQL table

In [ ]:
spark.sql("SELECT * FROM spam_stream").show()

+-----+--------------------+
|label|             message|
+-----+--------------------+
|  ham|I re-met alex nic...|
|  ham|K k :-):-) then w...|
|  ham|Are you being goo...|
|  ham|OH FUCK. JUSWOKE ...|
|  ham|K:)all the best:)...|
|  ham|I guess that's wh...|
| spam|-PLS STOP bootyde...|
|  ham|I'm tired of argu...|
|  ham|Everybody had fun...|
|  ham| Is ur lecture over?|
|  ham|Sounds better tha...|
|  ham|I‘ll have a look ...|
|  ham|It does it on its...|
| spam|Dear U've been in...|
|  ham|they released ano...|
|  ham|Jos ask if u wana...|
|  ham|Moji just informe...|
|  ham|Its like that hot...|
|  ham|Aight, text me to...|
|  ham|Its on in engalnd...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
(
    pipe_model
    .transform(spam_stream)
    .writeStream
    .format("memory")
    .trigger(processingTime="10 seconds")
    .queryName("predictions_stream")
    .start()
)

In [ ]:
spark.sql("select * from predictions_stream").show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+------------------------